In [10]:
import os
import sys
DIR_PATH = os.getcwd()
font_path = os.path.abspath(os.path.join(DIR_PATH, "../../src/frontend_functions" , "Fonts/BIZ-UDGothicB.ttc"))

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import datetime

def create_timetable_image(json_data, output_path="timetable.png", box_color="yellow"):
    # 画像の基本設定
    image_width = 400
    image_height = 1200
    margin = 50
    time_line_spacing = 90  # 縦pixel幅/30分ごと
    text_color = "black"
    background_color = "white"
    line_color = "gray"
    
    # タイムテーブルの時間範囲を計算
    time_format = "%H:%M"
    start_time = min(datetime.datetime.strptime(live["ライブステージ"]["from"], time_format) 
                     for live in json_data["タイムテーブル"])
    end_time = max(datetime.datetime.strptime(live["ライブステージ"]["to"], time_format) 
                   for live in json_data["タイムテーブル"])
    
    # タイムテーブル全体の時間範囲を30分刻みに拡張
    start_time = start_time.replace(minute=0)
    end_time = end_time.replace(minute=0) + datetime.timedelta(hours=1)
    total_minutes = int((end_time - start_time).total_seconds() / 60)
    
    # 画像の高さを動的に調整
    image_height = margin * 2 + (total_minutes // 30) * time_line_spacing
    
    # 画像を生成
    image = Image.new("RGB", (image_width, image_height), background_color)
    draw = ImageDraw.Draw(image)
    
    # フォントの設定（必要に応じて変更）
    try:
        font = ImageFont.truetype('\C:\Windows\Fonts\BIZ-UDGothicB.ttc', 15)
        font = ImageFont.truetype(font_path, 15)
        # font = ImageFont.truetype("arial.ttf", 20)
    except IOError:
        font = ImageFont.load_default()

    # 時間軸を描画
    current_y = margin
    current_time = start_time
    while current_time <= end_time:
        draw.line([(margin, current_y), (image_width - margin, current_y)], fill=line_color, width=1)
        draw.text((10, current_y - 10), current_time.strftime("%H:%M"), fill=text_color, font=font)
        current_y += time_line_spacing
        current_time += datetime.timedelta(minutes=30)
    
    # 各ライブ枠を描画
    for live in json_data["タイムテーブル"]:
        # 開始時間と終了時間を計算
        start = datetime.datetime.strptime(live["ライブステージ"]["from"], time_format)
        end = datetime.datetime.strptime(live["ライブステージ"]["to"], time_format)
        minutes = int((end-start).total_seconds() / 60)

        start_y = margin + int((start - start_time).total_seconds() / 60 / 30 * time_line_spacing)
        end_y = margin + int((end - start_time).total_seconds() / 60 / 30 * time_line_spacing)
        
        # 枠を描画
        draw.rectangle([(margin + 20, start_y), (image_width - margin - 20, end_y)], 
                       fill=box_color, outline="black")
        
        # テキストを描画
        try:
            artist_name = live["グループ名_採用"]
            if artist_name=="" or artist_name is None:
                artist_name = live["グループ名"]
        except KeyError:
            artist_name = live["グループ名"]

        time_text = f"{live['ライブステージ']['from']} ～ {live['ライブステージ']['to']} ({minutes})"
        text_x = margin + 30
        text_y = (start_y + end_y) // 2 - 20
        draw.text((text_x, text_y), artist_name, fill=text_color, font=font)
        draw.text((text_x, text_y + 20), time_text, fill=text_color, font=font)
    
    # 画像を保存
    image.save(output_path)
    print(f"タイムテーブル画像が作成されました: {output_path}")

In [12]:
import json
json_path = "../../data/projects/超明星現象2024/event_1/ライブ/stage_1.json"
with open(json_path, encoding="utf-8") as f:
    json_data = json.load(f)

# タイムテーブル画像を作成
create_timetable_image(json_data)

タイムテーブル画像が作成されました: timetable.png
